In [1]:
# !pip install transformers 
# !pip install protobuf==3.20.3

In [2]:
import sys


# Add the path to the wheel file
# Note: Update 'protobuf-downloader' to match the name of your notebook from Step 2
!pip install /kaggle/input/my-proto-wheel/protobuf-3.20.3-py2.py3-none-any.whl

Processing /kaggle/input/my-proto-wheel/protobuf-3.20.3-py2.py3-none-any.whl
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
tensorflow-metad

In [3]:
import pandas as pd
import os
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoImageProcessor, AutoModel

import torch.nn as nn
from tqdm import tqdm
import numpy as np

2025-11-19 11:39:15.584733: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763552355.867555      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763552355.959197      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
import warnings

warnings.filterwarnings('ignore')

# --- 1. Setup & Config ---
BASE_PATH = '/kaggle/input/csiro-biomass'
TEST_IMAGE_PATH = os.path.join(BASE_PATH)
TEST_META_PATH = os.path.join(BASE_PATH, 'test.csv')
 # Path to your saved model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



import cv2 
import pandas as pd
import numpy as np
import torch
import os
import joblib # For loading XGB/LGBM
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

MODEL_PATH = '/kaggle/input/dinov2/pytorch/giant/1' 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# --- LOAD DINO MODEL ---
print("Loading DINOv2...")
processor = AutoImageProcessor.from_pretrained(MODEL_PATH)
dino_model = AutoModel.from_pretrained(MODEL_PATH)
dino_model.to(device)
dino_model.eval()

# --- LOAD ML MODELS ---
print("Loading Ensemble Models...")
# Make sure these paths match where you saved them
xgb_model = joblib.load("/kaggle/input/baseline-dino-v3/models/lightgbm_ensemble.pkl")
lgb_model = joblib.load("/kaggle/input/baseline-dino-v3/models/xgboost_ensemble.pkl")

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Loading DINOv2...
Loading Ensemble Models...


In [5]:
# --- !! IMPORTANT: Use the CORRECT target columns !! ---
# These names are used to create the final submission file
TARGET_COLS =['Dry_Clover_g', 'Dry_Dead_g', 'Dry_Green_g', 'Dry_Total_g', 'GDM_g']

IMG_SIZE = 256 # Must be the same size you trained with

In [6]:
class BiomassTestDataset(Dataset):
    def __init__(self, df, base_path, processor, model):
        self.df = df
        self.image_paths = [os.path.join(base_path, p) for p in df['image_path']]
        self.processor = processor
        self.model = model
        self.device = model.device

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        
        # 1. Load Image
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # 2. Process Image for DINO
        inputs = self.processor(images=image, return_tensors="pt").to(self.device)
        
        # 3. Extract Features
        with torch.no_grad():
            outputs = self.model(**inputs)
            # Get the pooler_output and move to CPU immediately
            # Output shape from DINO is [1, 384]
            features = outputs.pooler_output.cpu()
            
        # Return the flattened feature vector [384,]
        return features.squeeze(0)

In [7]:
# --- Prepare DataLoader ---
test_df = pd.read_csv(TEST_META_PATH)

test_dataset = BiomassTestDataset(
    df=test_df,
    base_path=TEST_IMAGE_PATH,
    processor=processor,
    model=dino_model
)

test_loader = DataLoader(
    test_dataset,
    batch_size=32, 
    shuffle=False,
    num_workers=2
)

# --- Run Feature Extraction ---
all_features = []

print("Extracting features from Test set...")
with torch.no_grad():
    for feature_batch in tqdm(test_loader, desc="Extracting"):
        # feature_batch shape is [32, 384]
        all_features.append(feature_batch.numpy())

# Create the Final Feature Matrix
X_test = np.vstack(all_features)
print(f"Test Feature Shape: {X_test.shape}") # Should be (N_test_images, 384)

Extracting features from Test set...


Extracting: 100%|██████████| 1/1 [00:53<00:00, 53.84s/it]

Test Feature Shape: (5, 1536)


In [8]:
# print("Predicting with Ensemble...")

# # 1. Predict with XGBoost
# preds_xgb = xgb_model.predict(X_test)

# # 2. Predict with LightGBM
# preds_lgb = lgb_model.predict(X_test)

# # 3. Weighted Average (50/50)
# ensemble_preds = (0.5 * preds_xgb) + (0.5 * preds_lgb)

# # 4. Clip negative values (Biomass cannot be < 0)
# ensemble_preds = np.maximum(ensemble_preds, 0)

# print(f"Final Predictions Shape: {ensemble_preds.shape}")

In [9]:
N_TARGETS=len(TARGET_COLS)

In [10]:
META_LEARNER_COEFFS = np.array([
    [1.267, -0.217],  # Target 1
    [0.686, 0.262],   # Target 2
    [0.985, 0.211],   # Target 3
    [1.137, 0.003],   # Target 4
    [0.822, 0.372]    # Target 5
])

In [11]:
# --- PREDICTION STEP: APPLYING OPTIMAL WEIGHTS ---

print("\nPredicting with Optimal Ensemble Blend...")

# 1. Predict with XGBoost
preds_xgb = xgb_model.predict(X_test)

# 2. Predict with LightGBM
preds_lgb = lgb_model.predict(X_test)

# 3. Optimal Weighted Average (Target-Specific Stacking)
ensemble_preds_optimal = np.zeros_like(preds_xgb)

# Apply the learned optimal weights (coefficients) column by column
for i in range(N_TARGETS):
    xgb_weight = META_LEARNER_COEFFS[i, 0]
    lgbm_weight = META_LEARNER_COEFFS[i, 1]
    
    # Blend the predictions for the current target (column i)
    ensemble_preds_optimal[:, i] = (
        xgb_weight * preds_xgb[:, i] + 
        lgbm_weight * preds_lgb[:, i]
    )

# 4. Clip negative values (Biomass cannot be < 0)
ensemble_preds = np.maximum(ensemble_preds_optimal, 0)

print(f"Final Predictions Shape: {ensemble_preds.shape}")
# --- Create Submission File ---
print("\nCreating submission file...")


Predicting with Optimal Ensemble Blend...
Final Predictions Shape: (5, 5)

Creating submission file...


In [12]:

# --- Create Submission File ---
print("Creating submission file...")

# 1. Create a "wide" DataFrame with predictions
pred_df = pd.DataFrame(ensemble_preds, columns=TARGET_COLS)

# 2. Add the 'image_path' from the original test_df
submission_df = test_df[['image_path']].copy()
submission_df = pd.concat([submission_df, pred_df], axis=1)

# 3. "Melt" the DataFrame from "wide" to "long" format
submission_df_long = submission_df.melt(
    id_vars=['image_path'],
    value_vars=TARGET_COLS,
    var_name='target_name',
    value_name='target'
)

# 4. Create the 'sample_id'
# Clean up the filename (remove path and extension)
submission_df_long['image_name_base'] = submission_df_long['image_path'].apply(
    lambda p: os.path.splitext(os.path.basename(p))[0]
)

# Create ID: image_base + __ + target_name
submission_df_long['sample_id'] = submission_df_long['image_name_base'] + '__' + submission_df_long['target_name']

# 5. Filter duplicates and Select Columns
final_submission = submission_df_long[['sample_id', 'target']]
final_submission = final_submission.drop_duplicates(subset=['sample_id'])

# 6. Save
final_submission.to_csv('submission.csv', index=False)

print("submission.csv created successfully!")
print(final_submission.head())

Creating submission file...
submission.csv created successfully!
                     sample_id     target
0   ID1001187975__Dry_Clover_g   3.474077
5     ID1001187975__Dry_Dead_g  19.054260
10   ID1001187975__Dry_Green_g  24.684383
15   ID1001187975__Dry_Total_g  57.410470
20         ID1001187975__GDM_g  28.693285
